In [1]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

import time

#init display
from pynq.lib.arduino import Arduino_Displaycam
test = Arduino_Displaycam(base.ARDUINO)
test.init()
test.gpio()
test.pwm() 

0

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

from pynq.lib.arduino import Arduino_SPOT
tmp = Arduino_SPOT(base.ARDUINO,Arduino_SPOT)

In [3]:
buf0, buf1 = tmp.setImageAddress()

KeyboardInterrupt: 

In [4]:
start = time.time()
tmp.snapPic(35,0)
tmp.beginTX(1)
tmp.conversion()
tmp.beginCameraTransfer(4)
tmp.writeToTX(4, 'd')
end = time.time()
print(end-start)
print(tmp.microblaze.read_mailbox(0))
# for index in range(0,153600):
#     print(int((buf1[index].hex()), 16))

KeyboardInterrupt: 

In [8]:
tmp.beginTX(1)
tmp.writeToTX(4, "l36.5,94.5")
time.sleep(0.5)
tmp.writeToTX(4, "rR0495")
time.sleep(0.5)
tmp.writeToTX(4, "v0.15")

[]

In [3]:
#testing GPS 
seconds = 0
tmp.beginTX(1)
while(True):
    val = tmp.readFromGPS()
    print(val)
    time.sleep(2)
#     print(tmp.microblaze.read_mailbox(0,4))
#     time.sleep(1)
    if(val[0] != None and val[1] != None):
        tmp.writeToTX(4, "g," + str(round(val[0], 6)) + "," + str(round(val[1], 6)))
    print(seconds)
    seconds += 1
    

(None, None)
0
(None, None)
1
(34.414682666666664, -119.84186016666666)
2
(34.41467433333333, -119.84186933333334)
3
(34.41467316666667, -119.84187233333333)
4
(34.4146705, -119.8418695)
5
(34.41466616666666, -119.841865)
6


KeyboardInterrupt: 

In [ ]:
for i in range(0,230400,3):
    buf1[i] = bytes([0xff])
    buf1[i+1] = bytes([0x00])
    buf1[i+2] = bytes([0x00])
tmp.beginCameraTransfer(4)

In [4]:
import asyncio

@asyncio.coroutine
def flash_led(num):
    while True:
        yield from base.buttons[num].wait_for_value_async(1)
        if(num == 0):
            while base.buttons[num].read():
                base.leds[num].toggle()
                tmp.play_sequence([16,-100,16,-100,16,-50,16])
                time.sleep(10)
                tmp.stop()
                yield from asyncio.sleep(0.1)
            base.leds[num].off()
        elif(num == 1):
            while base.buttons[num].read():
                base.leds[num].toggle()
                tmp.snapPic(35,0)
                yield from asyncio.sleep(0.1)
            base.leds[num].off()

        elif(num == 2):
            while base.buttons[num].read():
                base.leds[num].toggle()
                print(tmp.get_gyro())
                print(tmp.get_compass())
                print(tmp.get_accl())
                print(tmp.get_euler())
                print(tmp.get_temperature())
                yield from asyncio.sleep(0.1)
            base.leds[num].off()
        else:
            while base.buttons[num].read():
                base.leds[num].toggle()
                seconds = 0
                while(seconds < 5):
                    values = tmp.range_poll()
                    print (values)
                    for val in values:
                        print(chr(val))
                    seconds += 1
                    time.sleep(1)
                yield from asyncio.sleep(0.1)
            base.leds[num].off()
        
        

In [5]:
tasks = [asyncio.ensure_future(flash_led(i)) for i in range(4)]

In [6]:
import psutil

@asyncio.coroutine
def print_cpu_usage():
    # Calculate the CPU utilisation by the amount of idle time
    # each CPU has had in three second intervals
    last_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
    while True:
        yield from asyncio.sleep(3)
        next_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
        usage = [(1-(c2-c1)/3) * 100 for c1,c2 in zip(last_idle, next_idle)]
        print("CPU Usage: {0:3.2f}%, {1:3.2f}%".format(*usage))
        last_idle = next_idle

tasks.append(asyncio.ensure_future(print_cpu_usage()))

In [7]:
if base.switches[0].read():
    print("Please set switch 0 low before running")
else:
    base.switches[0].wait_for_value(1)

CPU Usage: 1.00%, 0.00%
CPU Usage: -233.00%, 69.33%
CPU Usage: 0.67%, 0.00%
CPU Usage: 1.00%, 1.00%
CPU Usage: 0.67%, 0.00%
CPU Usage: 0.67%, 0.00%
CPU Usage: 0.00%, 0.33%
CPU Usage: 1.67%, 0.67%
CPU Usage: 0.00%, 0.33%
CPU Usage: 0.67%, 0.00%
CPU Usage: 0.67%, 0.00%
CPU Usage: 1.00%, 1.00%
CPU Usage: 0.00%, 0.33%


KeyboardInterrupt: 

In [19]:
[t.cancel() for t in tasks]

[True, True, True, True, True]

In [20]:
base.switches[0].wait_for_value(0)

In [3]:
#too fast?
tmp.init()

2

In [4]:
#works fine
tmp.gpio() 
tmp.pwm()

0

In [6]:
tmp.layerMode(1)
tmp.layerEffect(2)
tmp.layer(1)
time.sleep(0.05)
tmp.drawMainScreen()
tmp.layer(0)

In [12]:
print(tmp.getDevice())
tmp.beginTX(1)
for i in range(24):
    print(i, tmp.readRegister(i))

1
0 15
1 63
2 62
3 3
4 111
5 108
6 7
7 14
8 0
9 0
10 231
11 7
12 9
13 10
14 11
15 12
16 231
17 0
18 32
19 32
20 32
21 32
22 32
23 17


In [13]:
print(tmp.writeToTX(4,"Whats up"))

[]


In [4]:

tmp.clearWindow(1)

KeyboardInterrupt: 

In [7]:
#8bit yoga works
from PIL import Image
from numpy import array
img = Image.open('data/Yogananda.jpg')
# img.resize((240,240), Image.ANTIALIAS)
img_array = array(img)
img.close()

BIT = 0xb

file_size = 240*240
buf0 = tmp.buf_manager.cma_alloc(file_size, data_type="uint8_t")
buf1 = tmp.buf_manager.cma_get_buffer(buf0, file_size)
phy_addr = tmp.buf_manager.cma_get_phy_addr(buf0)
index = 0

for j in range(240):
    for i in range(240):
        red, green, blue = img_array[i][j]
        pixel = ((blue >> 6) | ((green & 0xE0) >> 3) | (red & 0xE0)) #rgb888 to rgb332
        buf1[index] = bytes([pixel]) 
        index += 1
tmp.microblaze.write_mailbox(0, 0)
tmp.microblaze.write_mailbox(4, 0)
tmp.microblaze.write_mailbox(8, phy_addr)
tmp.microblaze.write_mailbox(12, 240)
tmp.microblaze.write_mailbox(16, 240)
tmp.microblaze.write_blocking_command(0x6007)
print(tmp.microblaze.read_mailbox(0))

73


In [2]:
#testing image capture + drawing it pipelined!!!!
tmp.snapPic(35,0)

# stream_size = 153600
# buf0 = tmp.buf_manager.cma_alloc(stream_size, data_type="uint8_t")
# buf1 = tmp.buf_manager.cma_get_buffer(buf0, stream_size)
# phy_addr = tmp.buf_manager.cma_get_phy_addr(buf0)

# buf2 = tmp.buf_manager.cma_alloc(stream_size, data_type="uint8_t")
# buf3 = tmp.buf_manager.cma_get_buffer(buf2, stream_size)
# draw_addr = tmp.buf_manager.cma_get_phy_addr(buf2)

# tmp.microblaze.write_mailbox(0, phy_addr)
# tmp.microblaze.write_mailbox(4, draw_addr)
# tmp.microblaze.write_blocking_command(0x5000)

In [3]:
# testing haptic
tmp.play_sequence([16,-100,16,-100,16,-50,16])
time.sleep(10)
tmp.stop()

In [8]:
# testing IMU
import time 
i = 0
while i <= 60:
#     print(i, " seconds")
#     print(tmp.get_gyro())
#     print(tmp.get_compass())
#     print(tmp.get_accl())
    print(tmp.get_euler())
#     print(tmp.get_temperature())
    time.sleep(1)
    i = i + 1

[32.62, -4.94, 7.5]
[32.88, -4.81, 7.44]
[32.94, -5.69, 7.5]
[33.5, -5.25, 7.19]
[33.12, -5.75, 7.69]
[18.75, -7.69, 5.81]
[18.5, -6.62, 5.5]
[14.31, -6.38, 5.06]
[10.12, -4.44, 0.12]
[8.12, -1.94, -3.31]
[5.81, -5.12, -1.56]
[4.62, -11.25, -1.0]
[4.44, -12.69, -11.94]
[10.62, -6.5, -19.44]
[13.38, -5.06, -1.38]
[14.31, -3.19, -0.75]
[14.69, -3.5, -0.44]
[14.38, -3.25, -0.94]
[14.62, -3.12, -0.88]
[14.81, -3.25, -1.06]
[15.19, -3.0, -1.12]
[15.62, -2.88, -0.94]
[15.44, -2.56, -1.0]
[15.75, -3.5, -1.12]
[15.75, -4.75, -1.44]
[16.56, -1.06, -16.38]
[11.81, -1.81, -17.5]
[52.31, 26.62, -17.06]
[121.5, 2.69, 4.94]
[99.88, -0.19, 4.19]
[88.62, -0.44, 3.25]
[81.75, -0.44, 1.88]
[78.5, 1.0, 1.5]
[76.81, 7.25, -5.25]
[71.81, 5.81, -11.0]
[78.94, 3.75, -4.0]
[81.0, 3.0, -3.5]
[81.75, 2.62, -1.88]
[9.75, 0.62, -6.31]
[355.0, -3.31, -17.81]
[355.56, -44.44, -65.81]
[355.81, -47.38, -67.38]
[355.69, -48.75, -67.94]
[355.44, -49.12, -68.62]
[355.19, -49.62, -68.81]
[354.94, -49.88, -69.12]
[354.94,

In [14]:
i = 0
while i <= 60:
    print(i, " seconds")
    [ex, ey, ez] = tmp.get_euler()
    print(ex)
    time.sleep(1)
    i = i + 1
    if(ex > 350.0) :
        tmp.play_sequence([16])
        tmp.range_poll()
        time.sleep(1)
        i = i + 1
    if(ex < 10.0) :
        tmp.play_sequence([16])
        tmp.range_poll()
        time.sleep(1)
        i = i + 1
        
tmp.stop()
    


0  seconds
359.38
2  seconds
359.56
4  seconds
359.69
6  seconds
359.56
8  seconds
359.69
10  seconds
359.69
12  seconds
359.69
14  seconds
359.81
16  seconds
26.69
17  seconds
355.88
19  seconds
33.25
20  seconds
5.25
22  seconds
51.81
23  seconds
80.69
24  seconds
40.25
25  seconds
38.06
26  seconds
37.62
27  seconds
37.62
28  seconds
74.88
29  seconds
27.94
30  seconds
61.44
31  seconds
103.31
32  seconds
135.69
33  seconds
74.19
34  seconds
9.88
36  seconds
26.75
37  seconds
29.5


KeyboardInterrupt: 

In [7]:
# from statistics import median

# def parseRange(val):
#     returnString = ""
#     for x in val:
#         returnString += (chr(x))
#     return int(returnString[1:])

# #testing rangefinder
# count = 0
# medianList = []
# while(count < 20):
#     medianList.append(parseRange(tmp.range_poll()))
#     count += 1
# print(median(medianList))
# print(medianList)

print(tmp.medianRange())

54.0


In [3]:
tmp.beginTX(1)

def prepareToSend():
    gpsVal = parseGPS(tmp.readFromGPS())
    imuVal = parseIMU(tmp.get_euler())
    rangeVal = parseRange(tmp.range_poll())
    tag = "t,Big Rock!"
#     tmp.writeToTX(4, gpsVal)
    tmp.writeToTX(4, imuVal)
    tmp.writeToTX(4, rangeVal)
    tmp.writeToTX(4, tag)
    print(gpsVal, imuVal, rangeVal, tag)
    
def parseGPS(val):
    if(val[0] != None and val[1] != None):
        return "l," + str(round(val[0], 6)) + "," + str(round(val[1], 6))
    
def parseIMU(val):
    returnString = ""
    for x in val:
        returnString += (str(x) + ",")
    return "v," + returnString[0:-1]

def parseRange(val):
    returnString = ""
    for x in val:
        returnString += (str(x))
    return "r," + returnString[1:]

while(True):
    prepareToSend()
    time.sleep(5)

None v,285.5,2.19,-178.25 r,248485255 t,Big Rock!


KeyboardInterrupt: 

In [8]:
while(True):
    prepareToSend()
    time.sleep(1)

[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0025 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0002 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0025 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0002 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0025 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0002 t,Big Rock
[82, 48, 48, 50, 53]
0 t,Big Rock8,3.56,-179.06 r,25
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0025 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0020 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0025 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0000 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0025 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0000 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0025 t,Big Rock
[82, 48, 48, 50, 53]
None v,330.88,3.56,-179.06 r,0000 t,Big Rock
[82, 48, 48, 50, 53]
No

KeyboardInterrupt: 